In [ ]:
import os
import sys
from dotenv import find_dotenv
sys.path.append(os.path.dirname(find_dotenv()))

import numpy as np
import time as time
import glob
import random
import re
from collections import defaultdict

import constants

# Set up matplotlib and use a nicer set of plot parameters
%config InlineBackend.rc = {}
import matplotlib as mpl
mpl.rc('mathtext',fontset='stixsans')
mpl.rc('figure', facecolor="white")
#matplotlib.rc_file("../../templates/matplotlibrc")
import matplotlib.pyplot as plt
#import matplotlib.colors as colors
# %matplotlib notebook

VEGA_OUTPUT_DIR = os.path.join(constants.BIAS_DIR_BASE, 'xcorr', 'vega', 'output')

In [ ]:
regex = re.compile(r'.*bestfit_bias_survey_([^_]*)_mass_([^_]*)_([^_]*).*\.npy')

bias_bootstrap_dict = defaultdict(lambda: defaultdict(list))
chisq_dict_all = defaultdict(lambda: defaultdict(list))

In [ ]:
for p in glob.glob(f'{VEGA_OUTPUT_DIR}/*.npy'):
    match = regex.match(p)
    survey, logmass_lb, logmass_ub = match.group(1, 2, 3)
    logmass_lb, logmass_ub = float(logmass_lb), float(logmass_ub)
    bias, bias_err, reduced_chisq = np.load(p)
    bias_bootstrap_dict[survey][(logmass_lb, logmass_ub)].append(bias)
    chisq_dict_all[survey][(logmass_lb, logmass_ub)].append(reduced_chisq)

In [ ]:
figure_folder = os.path.join(constants.BIAS_DIR_BASE, 'figures')

for survey, bias_dict in bias_bootstrap_dict.items():
    chisq_dict = chisq_dict_all[survey]
    bin_boundaries = set(sum(bias_dict.keys(), ()))
    bin_boundaries = sorted(list(bin_boundaries))
    x = []
    bias_lower_bound = []
    bias_upper_bound = []
    reduced_chisq_lb = []
    reduced_chisq_ub = []
    for i in range(len(bin_boundaries) - 1):
        key = (bin_boundaries[i], bin_boundaries[i + 1])
        x.append(np.mean([bin_boundaries[i], bin_boundaries[i + 1]]))
        bias_list = bias_dict[key]
        assert bias_list
        bias_lower_bound.append(np.percentile(bias_list, 16))
        bias_upper_bound.append(np.percentile(bias_list, 84))
        reduced_chisq_lb.append(np.percentile(chisq_dict[key], 16))
        reduced_chisq_ub.append(np.percentile(chisq_dict[key], 84))
    plt.fill_between(x, bias_lower_bound, bias_upper_bound, color='black', alpha=0.2)
    plt.title(survey)
    plt.xlabel('Halo mass [M$_\odot$/h]')
    plt.ylabel('Galaxy bias')
    plt.tight_layout()
    plt.savefig(os.path.join(figure_folder, f'{survey}_bias_err.pdf'))
    plt.show()
        
    plt.fill_between(x, reduced_chisq_lb, reduced_chisq_ub, color='black', alpha=0.2)
    plt.title(survey)
    plt.xlabel('Halo mass [M_$\odot$/h]')
    plt.ylabel('Best-fit reduced chi-squared')
    plt.tight_layout()
    plt.savefig(os.path.join(figure_folder, f'{survey}_reduced_chisq.pdf'))
    plt.show()